**Wikipedia spanish language model** 

In [1]:
!pip install -q gwpy

     |████████████████████████████████| 1.4MB 13.6MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 2.7MB 55.4MB/s 


In [6]:
import zipfile
from google.colab import drive
from fastai.text import *
import os
import re

In [ ]:
drive.mount('/content/drive/')

In [ ]:
#to not show cell output-- too long
%%capture
!unzip -q "/content/drive/My Drive/Colab Notebooks/TLAC/wiki_art.zip"

In [12]:
def set_organization(dset):
  
  path="/content/wiki_art/{}".format(dset)

  for filename in os.listdir(path):
    os.rename(os.path.join(path, filename), os.path.join(path, re.sub('[^A-Za-z0-9]+(?![^.]*$)', '',filename)))

  n_files = len(list(os.listdir(path)))
  !ls $path
  print("elements_{}_set={} ".format(dset,n_files))


In [ ]:
set_organization("train")

In [ ]:
set_organization("test")

In [34]:
bs=16
txt_proc =[OpenFileProcessor(),
          TokenizeProcessor(tokenizer=Tokenizer(lang='es') ),
          NumericalizeProcessor(max_vocab=10000)]

#https://docs.fast.ai/text.data.html#TokenizeProcessor

In [35]:
from pathlib import Path

path = Path('/content/wiki_art')
path.ls()

[PosixPath('/content/wiki_art/test'), PosixPath('/content/wiki_art/train')]

In [ ]:
data_lm = (TextList.from_folder(path,processor=txt_proc)
          .filter_by_folder(include=['train', 'test']) 
          .split_by_rand_pct(0.1)
          .label_for_lm() #predicting the next word given the previous ones          
          .databunch(bs=bs))
data_lm.save('data_lm.pkl')

In [ ]:
data_lm.show_batch()

In [ ]:
data_lm = load_data(path_art, 'data_lm.pkl', bs=bs)